In [1]:
MODEL = "llama2"

In [12]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings()

# model.invoke("Tell me a joke?")

In [5]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Tell me a joke?")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [7]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("cv.pdf")

pages = loader.load_and_split()

pages

[Document(metadata={'source': 'cv.pdf', 'page': 0, 'page_label': '1'}, page_content='Sr. Python Engineer | Sr. Full Stack Engineer | Sr. Backend Developer \n \nEmail:    dontay.mobley.7p@outlook.com                 Contact Number: +1717 456 0046 \nLocation: Philadelphia, PA     Work Authorization: US citizen \n \nPROFESSIONAL SUMMARY \n\uf06c Over 10 years of experience in Design, Development and Deployment of Python applications. \n\uf06c Developed backend modules using Python on Django, Flask Web Framework using ORM models. \n\uf06c Developed ETL jobs in Python to extract and load data into various databases. \n\uf06c Used Test driven approach for developing the application and Implemented the unit tests using Python \nUnit test framework.  \n\uf06c Expertise in working with Perl and Shell scripting for designing automation framework. \n\uf06c Good experience in developing web applications and implementing Model View Control (MVC) \narchitecture using server - side applications like 

In [8]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't answer the question, reply "I don't know."

Context: {context}

Question: {question}

"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="Here is some context", question="What is your name?")


'\nAnswer the question based on the context below. If you can\'t answer the question, reply "I don\'t know."\n\nContext: Here is some context\n\nQuestion: What is your name?\n\n'

In [9]:
chain = prompt | model | parser

In [10]:
chain.invoke({
    "context": "The name I was given is Ali",
    "question": "What is your name?", 
})

' Of course! Based on the context you provided, the answer to the question "What is your name?" is "Ali".'

In [14]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    pages,
    embedding=embeddings
)

/home/syed-haider-ali-zaidi/PycharmProjects/LlamaLens/.venv/lib/python3.12/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [17]:
retriever = vectorstore.as_retriever()
retriever.invoke("Expertise in Machine Learning")

[Document(metadata={'source': 'cv.pdf', 'page': 4, 'page_label': '5'}, page_content='\uf0b7 Designed and maintained databases using Python and developed Python based API (RESTful Web \nService) using Flask, SQLAlchemy and PostgreSQL. \n\uf0b7 Implemented JSON Schema for validating the structure of JSON data. \n\uf0b7 Successfully migrated the Django database from SQLite to MySQL to PostgreSQL with complete data \nintegrity.  \n\uf0b7 Created Automation test framework using Selenium. \n\uf0b7 Implemented monitoring and established best practices around using elastic search. \n\uf0b7 Followed AGILE development methodology to develop the application. \n\uf0b7 Used Test driven approach (TDD) for developing services required for the application. \n \n \n \n \nEDUCATION \nUniversity of South Carolina                2007 – 2011 \nBachelor’s Degree in Computer Science'),
 Document(metadata={'source': 'cv.pdf', 'page': 0, 'page_label': '1'}, page_content='Sr. Python Engineer | Sr. Full Stack En

In [20]:
from operator import itemgetter
chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    } |
    prompt |
    model |
    parser
)

chain.invoke({
    "question": "Has he worked on Python?",
})

"Yes, based on his CV, he has worked extensively with Python. He has listed various projects and roles where he has used Python programming language, including:\n\n1. Senior Python Full Stack Engineer at Microsoft (Jun 2021 - Present)\n\t* Utilized Django Web Framework for creating web applications\n\t* Built REST API's using Django and integrated with Angular 14 on AWS service\n\t* Worked with Django user management modules to create authenticated application\n\t* Wrote Python scripts to parse JSON documents and load the data in PostgreSQL database\n2. Full Stack Developer at ABC Company (Jun 2019 - Jun 2021)\n\t* Developed experience in working with TDD/BDD environments and Scrum development lifecycle\n\t* Used Django API's to access the database\n\t* Documented the API's using tools like swagger\n3. Software Engineer at DEF Company (Jun 2017 - Jun 2019)\n\t* Worked with IDE's and also testing the API calls using postman before integrating with the front end\n\t* Experienced in docum

In [21]:
itemgetter("question")({"question": "What is your name?"})

'What is your name?'

In [ ]:
for s in chain.stream({"question": "How many years of experience do the candidate have?"}):
    print(s, end="", flush=True)